In [5]:
import firebase_admin
import re
import json
import time
import urllib.request
import os
from firebase_admin import credentials
from firebase_admin import db

In [6]:
#Firebase database 인증 및 앱 초기화
cred = credentials.Certificate(r'C:/Users/rlatn/Project_image_db/aicctv-8f5ac-firebase-adminsdk-n59g4-2e8482cb47.json')
firebase_admin.initialize_app(cred,{
    'databaseURL' : "https://aicctv-8f5ac.firebaseio.com/"
})


ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [7]:
ID = "00gpwls00" # 이용자마다 다르게 코딩해야 하는 부분
ref = db.reference(ID+"/VideoLink") #db 위치 지정

In [9]:
while True: # 무한 반복문
    
    dic=ref.get() # dict object
    tmp_str=json.dumps(dic, ensure_ascii=False)[1:-1] # 한글을 포함해 string으로 변환 후 맨 앞과 뒤의 {} 제거
    tmp=re.sub(r'"*"','', tmp_str) #  이름과 video linke만 남기고 나머지 문자 삭제

    video_nonsp_name = tmp.split(", ") # 사진 이름만 추출 
    video_name = []
    video_url = []
    for i in range(0, len(video_nonsp_name)):
        video_name.append(video_nonsp_name[i].split(": ")[0])
        video_url.append(video_nonsp_name[i].split(": ")[1])
    
    video_name = video_name[0:len(video_name)-2]
    video_url = video_url[0:len(video_url)-2]
    
    video_ref=db.reference(ID+"/VideoLink/") 
    Download_ref=video_ref.child('Download')     # 컴퓨터에 다운로드 된 마지막 영상의 이름
    Update_ref=video_ref.child('Update') # 데이터베이스에 업로드된 마지막 영상의 이름
        
    Download=Download_ref.get()
    Update=Update_ref.get()
        
    # 다운받은 마지막 영상의 이름과 서버에 update된 마지막 영상의 이름이 다른 경우
    while (Download != Update or Download=='0') : # 업데이트를 해야하는 경우 (다운로드가 필요한 경우)
        outpath="C:/"+ID+"/video/"
        # 위의 outpath 경로가 존재하지 않을 경우, 경로를 생성함
        if not os.path.isdir(outpath):
            os.makedirs(outpath)
        
        count = 0
        if(Download==0):
            count = -1
        else:
            for i in range(0, len(video_name)):
                if(video_name[i]==Download):
                    count = i

        for j in range (count+1, len(video_name)) : # 두 수의 차이만큼 반복 필요
            file_name=video_name[j]
            url=video_url[j]
            urllib.request.urlretrieve(url, outpath+file_name+".mp4") # 다운로드
            Download=file_name # 데이터베이스에 업데이트 하기 위해서 다운로드 받으며 체크
        Download_ref.set(Download) # 다운로드를 모두 완료한 후 변수값을 변경
            
    time.sleep(60); # 60초에 한번씩만 확인 및 다운로드


KeyboardInterrupt: 